In [10]:
%load_ext autoreload
%autoreload 2

from utils import get_args_parser, set_paths

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set directories

In [16]:
base_dir = '/Users/thomasbotch/Dropbox/Projects/vrGazeCore-Toolbox/python/data/'

subject_fn = 'furrow380_V1.txt'

# Set args to be parsed

In [34]:
args = [
    f'--project_dir={base_dir}',
    f'--raw_data_folder=rawData',
    f'--stim_folder=stimuli',
]

In [42]:
parser = get_args_parser()

params = parser.parse_args(args)
paths = set_paths(params)

Check that the following paths are correct:
project_raw_data_dir = /Users/thomasbotch/Dropbox/Projects/vrGazeCore-Toolbox/python/data/rawData
project_stim_dir = /Users/thomasbotch/Dropbox/Projects/vrGazeCore-Toolbox/python/data/stimuli
Are the paths correct?
1 = Yes
2 = No
Enter:1


In [190]:
vrGaze = vrGazeCore(params, paths)

raw_data = vrGaze.loadRawData(subject_fn)
parsed_data = vrGaze.parseRawData(raw_data)

In [191]:
test = vrGaze.confidenceFilter([parsed_data[0]])

Percent removed due to confidence: 0.80%


In [172]:
len(test[test[['LeftConf', 'RightConf']].mean(axis=1) > 0.6])

371

In [147]:
sum(test[['LeftConf', 'RightConf']].mean(axis=1) > 0.6)

371

Trial         Date         CoreTime    ExpTime  \
0       _sanityTarget360_0000   2022-01-24   10:20:36.29770   133.2174   
1       _sanityTarget360_0000   2022-01-24   10:20:36.31176   133.2302   
2       _sanityTarget360_0000   2022-01-24   10:20:36.32384   133.2441   
3       _sanityTarget360_0000   2022-01-24   10:20:36.33699   133.2565   
4       _sanityTarget360_0000   2022-01-24   10:20:36.35025   133.2699   
...                       ...          ...              ...        ...   
113079  _sanityTarget360_0000   2022-01-24   11:07:09.03745  1143.3150   
113080  _sanityTarget360_0000   2022-01-24   11:07:09.05193  1143.3280   
113081  _sanityTarget360_0000   2022-01-24   11:07:09.06452  1143.3420   
113082  _sanityTarget360_0000   2022-01-24   11:07:09.07805  1143.3550   
113083  _sanityTarget360_0000   2022-01-24   11:07:09.09253  1143.3690   

           Pitch       Yaw      Roll    RightX    RightY     LeftX     LeftY  \
0      -2.363129  5.323951 -3.704926  0.515778  0.466618  0.516094  0.456667   
1      -2.357422  5.330826 -3.716309  0.515313  0.466626  0.514224  0.456236   
2      -2.352966  5.340037 -3.723602  0.515313  0.466626  0.514224  0.456236   
3      -2.350494  5.346935 -3.727386  0.515325  0.466747  0.514006  0.457591   
4      -2.347473  5.355137 -3.730316  0.481729  0.467615  0.486136  0.454839   
...          ...       ...       ...       ...       ...       ...       ...   
113079 -0.967194 -1.807465 -1.665161  0.501687  0.510090  0.503872  0.511875   
113080 -0.955200 -1.799255 -1.643066  0.501687  0.510090  0.503872  0.511875   
113081 -0.945038 -1.793396 -1.616943  0.501590  0.510316  0.503885  0.511986   
113082 -0.935272 -1.787720 -1.591644  0.501590  0.510316  0.503885  0.511986   
113083 -0.926361 -1.791840 -1.570404  0.501027  0.509759  0.502875  0.512853   

        RightConf  LeftConf  Trial Rotation  
0             1.0       1.0               0  
1             1.0       1.0               0  
2             1.0       1.0               0  
3             1.0       1.0               0  
4             1.0       1.0               0  
...           ...       ...             ...  
113079        1.0       1.0               0  
113080        1.0       1.0               0  
113081        1.0       1.0               0  
113082        1.0       1.0               0  
113083        1.0       1.0               0  

[113084 rows x 14 columns]

In [189]:
import os
import pandas as pd
import numpy as np
from datetime import datetime as dt

class vrGazeCore:

    def __init__(self, params, paths):
        # initialize the data class using 
        self.params = params
        self.paths = paths


    def returnHead(self):
        print(self.data.head())


    def returnListItem(self, num):
        #return frame item at index num
        print(self.data[num].head())


    def loadRawData(self, filename, header=None):
        
        # Set the path to the file based on our paths dictionary and load using pandas
        file_path = os.path.join(self.paths['project_raw_data_dir'], filename)
        raw_data = pd.read_csv(file_path, sep = ",", header=header)
        
        # Add default header labels
        if header is None:
            column_labels = ['Trial', 'Date', 'CoreTime', 'ExpTime', 'Yaw', 'Pitch', 
                'Roll', 'RightX', 'RightY', 'LeftX', 'LeftY', 'RightConf', 'LeftConf', 'Trial Rotation']
            
            raw_data.columns = column_labels 
            
        # Set the core
#         raw_data['CoreTime'] = raw_data['CoreTime'].apply(lambda x: dt.strptime(x.strip(), '%H:%M:%S.%f').microsecond)
        
        return raw_data
    
    def processRawData(self, raw_data):
        
        # these are the basic columns we need for head fixations
        columns = ['ExpTime', 'Yaw', 'Pitch', 'Roll']
        
        #### Head-Tracking Data ####
        if params.headset_type == 3:
            return raw_data[columns]
        
        #### Eye-Tracking Data ####
        
        # Right: eye coordinates + confidence
        if params.use_eye == 0:
            columns.extend(['RightX', 'RightY', 'RightConf'])
            return raw_data[columns]
        
        # Left: eye coordinates + confidence
        elif params.use_eye == 1:
            columns.extend(['LeftX', 'LeftY', 'LeftConf'])
            return raw_data[columns]
        
        # Best: left or right eye coordinates based on best confidence
        elif params.use_eye == 2:
            columns.extend(['RightX', 'RightY', 'LeftX', 'LeftY', 'RightConf', 'LeftConf'])
        
        # Average: average of left/right coordinates + minimum of confidence
        else:
            columns.extend(['RightX', 'RightY', 'LeftX', 'LeftY', 'RightConf', 'LeftConf'])
            return raw_data[columns]
            
        
        
#         # Vive Eye 2D tracking needs to be normalized to same field as pupil
#         if params.headset_type == 2 and params.gaze_type == 0:
#             parData(:,5:6) = (parData(:,5:6) + 1)/2; % bring positive, then half
            
    def parseRawData(self, raw_data):
        
        #create a row comparison for the dataframe, then split into trials based on row comparison
        def comp_prev(trial):
            return np.concatenate(([False], trial[1:] == trial[:-1]))

        trial_changes = comp_prev(raw_data.Trial.values)

        #split into multiple dataframes based on trial
        parsed_data = np.split(raw_data, np.where(trial_changes == 0)[0])[1:] 
        
        return parsed_data


    def confidenceFilter(self, df_list, threshold=0.6):

        """

        This function is used for filtering scene data by the confidence threshold. The default
        requires a confidence of 0.6 to keep a data point, otherwise the point will be removed.

        By default, the threshold will use the average of the two eyes confidence (eyeType=2). The 
        following options are supported:
            - eyeType=0: right-eye only
            - eyeType=1: left-eye only

        """

        if self.params.use_eye == 0:
            avg_type = ['RightConf']
        elif self.params.use_eye == 1:
            avg_type = ['LeftConf']
        else:
            avg_type = ['RightConf', 'LeftConf']

        for trial in df_list: #go through each dataframe --> trial will be a dataframe of individual trial
            len_pre_threshold = len(trial)
            trial = trial[trial[avg_type].mean(axis=1) > threshold] # remove datapoint in the trial that fell below confidence threshold
            len_post_threshold = len(trial)
            
            percent_removed = 100*(len_pre_threshold - len_post_threshold) / len_pre_threshold
            print (f"Percent removed due to confidence: {percent_removed:.2f}%")
            
        return df_list


    def eccentricityFilter(self):

        """

        Remove any datapoints with eye-tracking coords beyond a given degree.

        tlb ---> figure this out

        """

    def calculateTrialFPS(self):

        """

        Calculates each trial's FPS with both the computer's core clock number and the Unity 
        Engine numbers

        """

        fpsArr = np.full([len(self.data),3], np.nan)
        trialNum = 0;

        for trial in self.data:

            coreFPS = np.mean(1.0/trial['CoreTime'].diff()) #FPS calculation 1second / how close one frame happened to another on average
            unityFPS = np.mean(1.0/trial['ExpTime'].diff())

            trialName = trial['Trial'][0]

            fpsArr[trialNum] = np.where(fpsArr[trialNum], [trialName, coreFPS, unityFPS], 0)
            trialNum += 1

        return pd.DataFrame(fpsArr, columns=['Trial Name', 'Core FPS', 'Unity FPS'])


